### Imports

In [2]:
from google.colab import drive
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import os
import numpy as np
import random
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import train_test_split

In [3]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Set random seeds for reproducibility
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [5]:
# Define the base path to your folders
base_path = '/content/drive/My Drive/ANNDL/'

# Define the full paths to the folders
healthy_path = os.path.join(base_path, 'public_healthy')
unhealthy_path = os.path.join(base_path, 'public_unhealthy')

In [6]:
# Function to load images and convert them to numpy arrays
def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        try:
            with Image.open(img_path) as img:
                images.append(np.array(img))
        except IOError:
            print(f"Cannot open {filename}")
    return np.array(images)

In [7]:
# Load all images from both folders into numpy arrays
plants_healthy = load_images_from_folder(healthy_path)
plants_unhealthy = load_images_from_folder(unhealthy_path)

In [8]:
# Merge plant datasets
X = np.concatenate([plants_healthy, plants_unhealthy], axis=0)

# Create labels: 0 for 'healthy', 1 for 'unhealthy'
y = np.concatenate([np.zeros(len(plants_healthy)), np.ones(len(plants_unhealthy))], axis=0)

# One-hot encode labels
y = tf.keras.utils.to_categorical(y, num_classes=2)

# Split into train and validation sets
val_size = 0.2
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=val_size, stratify=np.argmax(y, axis=1), random_state=seed)


In [9]:
# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
datagen.fit(X_train)

In [10]:
# DenseNet Model Setup
densenet = DenseNet121(include_top=False, weights='imagenet', input_shape=X_train.shape[1:], pooling='avg')
densenet.trainable = False  # Freeze DenseNet layers initially

inputs = Input(shape=X_train.shape[1:])
x = densenet(inputs)
x = BatchNormalization()(x)  # Add Batch Normalization
x = Dropout(0.5)(x)  # Add Dropout
outputs = Dense(2, activation='softmax')(x)
model = Model(inputs, outputs)


29084464/29084464 [==============================] - 0s 0us/step


In [11]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [12]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('SubmissionModel', save_best_only=True)
reduce_lr = ReduceLROnPlateau(factor=0.2, patience=5)


In [13]:
# Initial Training
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32, seed=seed),
    epochs=1000,  # Set a suitable number of epochs
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, model_checkpoint, reduce_lr]
)

Epoch 1/1000
125/125 [==============================] - 75s 472ms/step - loss: 0.8169 - accuracy: 0.6050 - val_loss: 0.6787 - val_accuracy: 0.6360 - lr: 0.0010
Epoch 2/1000
125/125 [==============================] - 53s 429ms/step - loss: 0.6701 - accuracy: 0.6478 - val_loss: 0.6020 - val_accuracy: 0.6760 - lr: 0.0010
Epoch 3/1000
125/125 [==============================] - 56s 453ms/step - loss: 0.6465 - accuracy: 0.6645 - val_loss: 0.5861 - val_accuracy: 0.6870 - lr: 0.0010
Epoch 4/1000
125/125 [==============================] - 14s 115ms/step - loss: 0.6244 - accuracy: 0.6768 - val_loss: 0.5870 - val_accuracy: 0.6740 - lr: 0.0010
Epoch 5/1000
125/125 [==============================] - 54s 437ms/step - loss: 0.6024 - accuracy: 0.6900 - val_loss: 0.5511 - val_accuracy: 0.7190 - lr: 0.0010
Epoch 6/1000
125/125 [==============================] - 15s 118ms/step - loss: 0.5983 - accuracy: 0.6855 - val_loss: 0.5546 - val_accuracy: 0.7210 - lr: 0.0010
Epoch 7/1000
125/125 [==================

In [14]:
# Fine-Tuning
densenet.trainable = True
fine_tune_at = 100
for layer in densenet.layers[:fine_tune_at]:
    layer.trainable = False


In [15]:
# Recompile the model with a lower learning rate
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=1e-5), metrics=['accuracy'])


In [16]:
# Continue Training for Fine-Tuning
history_fine = model.fit(
    datagen.flow(X_train, y_train, batch_size=32, seed=seed),
    epochs=1000,  # Adjust as necessary
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, model_checkpoint, reduce_lr]
)
# Save the model
model.save('SubmissionModel.h5')
# Save the model
model.save('SubmissionModel')

Epoch 1/1000
125/125 [==============================] - 80s 226ms/step - loss: 0.5681 - accuracy: 0.7340 - val_loss: 23.8002 - val_accuracy: 0.7280 - lr: 0.0010
Epoch 2/1000
125/125 [==============================] - 19s 148ms/step - loss: 0.4672 - accuracy: 0.7925 - val_loss: 0.7582 - val_accuracy: 0.7560 - lr: 0.0010
Epoch 3/1000
125/125 [==============================] - 18s 146ms/step - loss: 0.4214 - accuracy: 0.8077 - val_loss: 0.9185 - val_accuracy: 0.6960 - lr: 0.0010
Epoch 4/1000
125/125 [==============================] - 67s 542ms/step - loss: 0.3819 - accuracy: 0.8313 - val_loss: 0.3926 - val_accuracy: 0.8230 - lr: 0.0010
Epoch 5/1000
125/125 [==============================] - 19s 151ms/step - loss: 0.3681 - accuracy: 0.8380 - val_loss: 0.5661 - val_accuracy: 0.7570 - lr: 0.0010
Epoch 6/1000
125/125 [==============================] - 19s 151ms/step - loss: 0.3792 - accuracy: 0.8378 - val_loss: 0.7277 - val_accuracy: 0.8570 - lr: 0.0010
Epoch 7/1000
125/125 [=================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
# Save the model
model.save('SubmissionModel.h5')
# Save the model
model.save('SubmissionModel')
model.save(os.path.join('/content/drive/My Drive/ANN & DL/', 'SubmissionModel'))

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
# Evaluate the model
def evaluate_model(history):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy over epochs')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss over epochs')
    plt.legend()
    plt.show()

evaluate_model(history)

NameError: ignored

In [ ]:
# Validation accuracy
val_pred = np.argmax(model.predict(X_val), axis=1)
val_true = np.argmax(y_val, axis=1)
print('Validation accuracy:', accuracy_score(val_true, val_pred))